In [1]:
import numpy as np
import matplotlib.pyplot as plt

#import tensorflow_datasets as tfds
import tensorflow as tf
import pickle
import skrf as rf
from scipy.spatial import distance
from scipy.linalg import eigh
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import torch.optim as optim

import math, random, functools
from typing import Optional, Dict, Tuple
from torch.utils.data import Dataset, DataLoader

2025-08-20 23:25:08.045310: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755750308.067725 3789183 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755750308.075321 3789183 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-20 23:25:08.103482: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
#define the targeted max frequency
fmax = 200
#define function for loading ind and s-parameters
#"path" is the path for ind of transformer layout
#"label" is the path for labels. In this case, labels are S-parameter associated with each ind 
def load_ind_from_path_MxN(path, label,length):
    ind_a = []
    ind_b = []
    labels = []
    error = []
    for kk in range(length):
        #load ind
        ind = np.load(path+str(kk)+'.npz')
        result_a = []
        result_b = []
        dst_a = np.zeros(len(ind['a'])-1)
        aj_a = np.zeros((len(ind['a']),len(ind['a'])))
        dst_b = np.zeros(len(ind['b'])-1)
        aj_b = np.zeros((len(ind['b']),len(ind['b'])))
        
        for x in range(len(ind['a'])-1):
            dst = distance.euclidean(ind['a'][x],ind['a'][x+1])
            dst_a[x] = dst
            aj_a[x][x+1] = dst_a[x]
            aj_a[x+1][x] = dst_a[x]
        
        eigenvalues, eigenvectors = eigh(aj_a)
        
        for x in range(int(np.ceil(len(ind['a'])/2))):
            result_a.append([ind['a'][x][0]/100,ind['a'][x][1]/100,ind['a'][x+1][0]/100,ind['a'][x+1][1]/100,dst_a[x]/100,0,np.sum(eigenvectors[x][1]+eigenvectors[x+1][1])])
        ind_a.append(result_a)
        
        for x in range(len(ind['b'])-1):
            dst = distance.euclidean(ind['b'][x],ind['b'][x+1])
            dst_b[x] = dst
            aj_b[x][x+1] = dst_b[x]
            aj_b[x+1][x] = dst_b[x]
        
        eigenvalues, eigenvectors = eigh(aj_b)
        
        for x in range(int(np.ceil(len(ind['b'])/2))):
            result_b.append([ind['b'][x][0]/100,ind['b'][x][1]/100,ind['b'][x+1][0]/100,ind['b'][x+1][1]/100,dst_b[x]/100,0,np.sum(eigenvectors[x][1]+eigenvectors[x+1][1])])
        ind_b.append(result_b)
        #load S-parameters
        #According to symmerty, only S11,S12,S13,S14,S33,S34 are considered
        #Each S-parameter has real and imaginary parts, and hence 12 real values in total.
        results = []
        datafile = label+str(kk)+".s4p"
        spt = rf.Network(datafile)
        step = int(1e9/(spt.f[1]-spt.f[0]))
        for k in range(fmax+1):
            i = int(k*step/2)
            results.append([spt.s[i][0][0].real,spt.s[i][0][0].imag,
                            spt.s[i][0][1].real,spt.s[i][0][1].imag,
                            spt.s[i][0][2].real,spt.s[i][0][2].imag,
                            spt.s[i][0][3].real,spt.s[i][0][3].imag,
                            spt.s[i][2][2].real,spt.s[i][2][2].imag,
                            spt.s[i][2][3].real,spt.s[i][2][3].imag])     
        sp = np.array(results)
        invalid = False
        '''
        for k in range(len(sp)-3):
            for x in range(12):
                if abs(sp[k][x]+sp[k+2][x]-2*sp[k+1][x])>0.03:
                    invalid = True
                    '''
        if (invalid):
            error.append(kk)
        labels.append(results) 
    return ind_a,ind_b, labels,error

In [3]:
def load_ind_from_path(path, label,length):
    ind_a = []
    ind_b = []
    labels = []
    error = []
    for kk in range(length):
        result_a = []
        result_b = []
        #load ind
        file_path = (path+str(kk)+'.pkl')
        with open(file_path, 'rb') as f:
            loaded_list = pickle.load(f)
        totalD = loaded_list[0]
        node_D = []
        for item in totalD:
            wid = item[0]
            layer = item[1]
            for node in item[2]:
                node_D.append(node)
        #print(np.shape(node_D))
        node_D = list(dict.fromkeys(node_D))
        #print(np.shape(node_D))
        inda_node = np.array(node_D)
        #print(np.shape(inda_node))
        totalU = loaded_list[1]
        node_U = []
        for item in totalD:
            wid = item[0]
            layer = item[1]
            for node in item[2]:
                node_U.append(node)
        node_U = list(dict.fromkeys(node_U))
        indb_node = np.array(node_U)
        dst_a = np.zeros(len(inda_node)-1)
        aj_a = np.zeros((len(inda_node),len(inda_node)))
        dst_b = np.zeros(len(indb_node)-1)
        aj_b = np.zeros((len(indb_node),len(indb_node)))
        
        for x in range(len(inda_node)-1):
            dst = distance.euclidean(inda_node[x],inda_node[x+1])
            dst_a[x] = dst
            aj_a[x][x+1] = dst_a[x]
            aj_a[x+1][x] = dst_a[x]
        
        eigenvalues, eigenvectors = eigh(aj_a)
        
        for x in range(len(inda_node)-1):
            if(inda_node[x+1][0]==15) or (inda_node[x+1][0]==-15):
                continue
            result_a.append([inda_node[x][0]/100,inda_node[x][1]/100,inda_node[x+1][0]/100,inda_node[x+1][1]/100,dst_a[x]/100,0,np.sum(eigenvectors[x][1]+eigenvectors[x+1][1])])
        ind_a.append(result_a)
        
        for x in range(len(indb_node)-1):
            dst = distance.euclidean(indb_node[x],indb_node[x+1])
            dst_b[x] = dst
            aj_b[x][x+1] = dst_b[x]
            aj_b[x+1][x] = dst_b[x]
        
        eigenvalues, eigenvectors = eigh(aj_b)
        
        for x in range(len(indb_node)-1):
            if(indb_node[x+1][0]==15) or (indb_node[x+1][0]==-15):
                continue
            result_b.append([indb_node[x][0]/100,indb_node[x][1]/100,indb_node[x+1][0]/100,indb_node[x+1][1]/100,dst_b[x]/100,0,np.sum(eigenvectors[x][1]+eigenvectors[x+1][1])])
        ind_b.append(result_b)
        #load S-parameters
        #According to symmerty, only S11,S12,S13,S14,S33,S34 are considered
        #Each S-parameter has real and imaginary parts, and hence 12 real values in total.
        results = []
        datafile = label+str(kk)+".s4p"
        spt = rf.Network(datafile)
        step = int(1e9/(spt.f[1]-spt.f[0]))
        for k in range(fmax+1):
            i = k*step
            results.append([spt.s[i][0][0].real,spt.s[i][0][0].imag,
                            spt.s[i][0][1].real,spt.s[i][0][1].imag,
                            spt.s[i][0][2].real,spt.s[i][0][2].imag,
                            spt.s[i][0][3].real,spt.s[i][0][3].imag,
                            spt.s[i][2][2].real,spt.s[i][2][2].imag,
                            spt.s[i][2][3].real,spt.s[i][2][3].imag])     
        sp = np.array(results)
        invalid = False
        '''
        for k in range(len(sp)-3):
            for x in range(12):
                if abs(sp[k][x]+sp[k+2][x]-2*sp[k+1][x])>0.03:
                    invalid = True
                    '''
        if (invalid):
            error.append(kk)
        labels.append(results) 
    return ind_a,ind_b, labels,error

In [4]:
def load_from_path_MxN(path):
    logfile = open(path+"log.txt","r")
    listall = list(map(lambda item: list(map(lambda jtem: eval(jtem.strip()), item[1:-1].split(','))), logfile.read().strip().split('\n')))
    tabular_data = np.array(listall)

#load images and s-parameters
    length = len(tabular_data)
    ind_a, ind_b,labels,error = load_ind_from_path_MxN(path+"SEG/", path+"SPData/",length)

    #show_images(images)
    srf = np.load(path+"/srf.npy")

    for kk in range(length):
        if kk in error:
            continue  
    #select 1 turn and 1 turn transformers
        if((tabular_data[kk,2]!=0) or (tabular_data[kk,3]!=0)):
       #b only includes the variable parameters in our designs. 
       #Constants at this design stages are excluded
            for i in range(len(ind_a[kk])):
                ind_a[kk][i][5] = tabular_data[kk][6]
            for i in range(len(ind_b[kk])):
                ind_b[kk][i][5] = tabular_data[kk][7]
            ind_a11.append(np.array(ind_a[kk]))
            ind_b11.append(np.array(ind_b[kk]))
            labels11.append(labels[kk])
            srf_list.append(srf[kk])

In [5]:
def load_from_path_parallel(path):
    logfile = open(path+"log.txt","r")
    listall = list(map(lambda item: list(map(lambda jtem: eval(jtem.strip()), item[1:-1].split(','))), logfile.read().strip().split('\n')))
    tabular_data = np.array(listall)

#load images and s-parameters
    length = len(tabular_data)
    ind_a, ind_b,labels,error = load_ind_from_path(path+"SEG/", path+"SPData/",length)

    #show_images(images)
    srf = np.load('/rdf/shared/design_automation/Data_EMX/XFMRParallel_2504/srf.npy')
    for kk in range(length):
        if kk in error:
            continue  
        #select 1 turn and 1 turn transformers
        if((tabular_data[kk,4]!=0) and (tabular_data[kk,5]!=0)):
           #b only includes the variable parameters in our designs. 
           #Constants at this design stages are excluded
            for i in range(len(ind_a[kk])):
                ind_a[kk][i][5] = tabular_data[kk][4]
            for i in range(len(ind_b[kk])):
                ind_b[kk][i][5] = tabular_data[kk][4]
            ind_a11.append(np.array(ind_a[kk]))
            ind_b11.append(np.array(ind_b[kk]))
            labels11.append(labels[kk])
            srf_list.append(srf[kk])

In [6]:
def load_from_path_8shaped(path):
    logfile = open(path+"log.txt","r")
    listall = list(map(lambda item: list(map(lambda jtem: eval(jtem.strip()), item[1:-1].split(','))), logfile.read().strip().split('\n')))
    tabular_data = np.array(listall)

#load images and s-parameters
    length = len(tabular_data)
    ind_a, ind_b,labels,error = load_ind_from_path(path+"SEG/", path+"SPData/",length)

    #show_images(images)
    srf = np.load('/rdf/shared/design_automation/Data_EMX/XFMR8Shaped1x2_2504/srf.npy')
    for kk in range(length):
        if kk in error:
            continue  
        #select 1 turn and 1 turn transformers
        if((tabular_data[kk,4]!=0) and (tabular_data[kk,5]!=0)):
           #b only includes the variable parameters in our designs. 
           #Constants at this design stages are excluded
            for i in range(len(ind_a[kk])):
                ind_a[kk][i][5] = tabular_data[kk][4]
            for i in range(len(ind_b[kk])):
                ind_b[kk][i][5] = tabular_data[kk][4]
            ind_a11.append(np.array(ind_a[kk]))
            ind_b11.append(np.array(ind_b[kk]))
            labels11.append(labels[kk])
            srf_list.append(srf[kk])

In [7]:
ind_a11 = []
ind_b11 = []
labels11 = []
srf_list = []
for i in range(4):
    for j in range(4):
        path = "/rdf/shared/design_automation/Data_EMX/XFMR_2503_"+str(i+1)+"x"+str(j+1)+"/"
        load_from_path_MxN(path)

path = '/rdf/shared/design_automation/Data_EMX/XFMRParallel_2504/'
load_from_path_parallel(path)

path = '/rdf/shared/design_automation/Data_EMX/XFMR8Shaped1x2_2504/'
load_from_path_8shaped(path)

### Model

In [8]:
# Set hyperparameters
d_model = 32
num_heads = 4
dff = 256
num_layers = 2
max_seq_len = 32
height = 7
dropout_rate = 0.1


In [9]:
#################################
# Positional Encoding
#################################
class PositionalEncoding(nn.Module):
    def __init__(self, seq_len, d_model):
        super(PositionalEncoding, self).__init__()
        positions = torch.arange(seq_len).unsqueeze(1).float()  # [seq_len, 1]
        dims = torch.arange(d_model).unsqueeze(0).float()  # [1, d_model]

        angle_rates = 1 / torch.pow(10000.0, (2 * (dims // 2)) / d_model)
        angle_rads = positions * angle_rates

        # Compute sine and cosine
        sines = torch.sin(angle_rads[:, 0::2])
        cosines = torch.cos(angle_rads[:, 1::2])

        # Concatenate sine and cosine
        pos_encoding = torch.cat([sines, cosines], dim=-1)  # [seq_len, d_model]
        self.pos_encoding = pos_encoding.unsqueeze(0)  # [1, seq_len, d_model]

    def forward(self, x):
        return self.pos_encoding[:, :x.size(1), :].to(x.device)

#################################
# Padding Mask
#################################
def create_padding_mask(x):
    """
    Compute the padding mask:
    - x: [batch_size, seq_len, height]
    - mask: [batch_size, 1, 1, seq_len]
    """
    summed = torch.sum(torch.abs(x), dim=2)  # [batch_size, seq_len]
    mask = (summed == 0).float()  # 1.0 indicates padding
     # Expand mask to match MultiheadAttention expected shape
    mask = mask[:, None, None, :]  # Shape: [batch_size, 1, 1, seq_len]
    
    # Reshape to [batch_size * num_heads, seq_len, seq_len]
    batch_size, seq_len, height = x.shape
    mask = mask.expand(batch_size, num_heads, seq_len, seq_len)  # [batch_size, num_heads, seq_len, seq_len] 
    # Merge batch_size and num_heads into the first dimension
    mask = mask.reshape(batch_size * num_heads, seq_len, seq_len)
    return mask


#################################
# Example Usage
#################################
# Creating a batch of variable-length sequences with each element being a vector of length 'height'.
# For demonstration:
# Sequence 1: length=4
# Sequence 2: length=7
# Sequence 3: length=3
# We'll pad them to length=7.

def pad_sequence(seq_len, max_len=7):
    length = seq_len.shape[0]
    pad_len = max_len - length
    return torch.cat([seq_len, torch.zeros(pad_len, height)], dim=0)

In [10]:
max_len = max(len(seq) for seq in ind_b11)
max_len = int(max_len) 

new_a = []
for a in ind_a11:
    padded_a = pad_sequence(torch.tensor(a, dtype=torch.float32),max_len)
    new_a.append(padded_a.detach().numpy())
#transformer_encoder.eval()  # Set to evaluation mode
#transformer_encoder = TransformerEncoder(num_layers, d_model, num_heads, dff, max_seq_len)
#output = transformer_encoder(torch.stack(new_a))
#print("Output shape:", output.shape)  # [batch_size, seq_len, d_model]
new_b = []
for b in ind_b11:
    padded_b = pad_sequence(torch.tensor(b, dtype=torch.float32),max_len)
    new_b.append(padded_b.detach().numpy())

class item:
  def __init__(self, a,b,label,srf):
    self.a = a
    self.b = b
    self.label = label
    self.srf = srf
data = []
for i in range(len(ind_a11)):
    data.append(item(new_a[i],new_b[i],labels11[i],srf_list[i]))

np.random.shuffle(data)

length = len(ind_a11)
split1 = int(0.6*length)
split2 = int(0.8*length)
data_train = data[0:split1]
data_valid = data[split1:split2]
data_test = data[split2:]

#Separate the training, validation and test datasets 
train_a = []
train_labels = []
train_b = []
train_srf = []
valid_a = []
valid_labels = []
valid_b = []
valid_srf = []
test_a = []
test_labels = []
test_b = []
test_srf = []

#The targeted frequency span up to fmax, with 1GHz step
s_max = np.zeros((fmax,12))
s_min = np.zeros((fmax,12))
srange = np.zeros((fmax,12))


for i in range(len(data_train)):
    train_a.append(data_train[i].a)
    train_b.append(data_train[i].b)
    train_labels.append(data_train[i].label)
    train_srf.append(data_train[i].srf)
x_train_a = np.array(train_a) 
x_train_b = np.array(train_b)
x_train_srf = np.array(train_srf)
y_train_encoded = np.array(train_labels)[:,1:fmax+1,:]


for i in range(len(data_valid)):
    valid_a.append(data_valid[i].a)
    valid_b.append(data_valid[i].b)
    valid_labels.append(data_valid[i].label)
    valid_srf.append(data_valid[i].srf)
x_valid_a = np.array(valid_a) 
x_valid_b = np.array(valid_b)
x_valid_srf = np.array(valid_srf)
y_valid_encoded = np.array(valid_labels)[:,1:fmax+1,:]


for i in range(len(data_test)):
    test_a.append(data_test[i].a)
    test_b.append(data_test[i].b)
    test_labels.append(data_test[i].label)
    test_srf.append(data_test[i].srf)
x_test_a = np.array(test_a) 
x_test_b = np.array(test_b)
x_test_srf = np.array(test_srf)
y_test_encoded = np.array(test_labels)[:,1:fmax+1,:]

#normalize the s-parameters for each frequency point
for z in range(fmax):
    for i in range(12):
        s_max[z,i] = max(max(y_train_encoded[:,z,i]),max(y_valid_encoded[:,z,i]),max(y_test_encoded[:,z,i]))
        s_min[z,i] = min(min(y_train_encoded[:,z,i]),min(y_valid_encoded[:,z,i]),min(y_test_encoded[:,z,i]))
        srange[z,i] = s_max[z,i]-s_min[z,i]
    
    for i in range(12):
        y_train_encoded[:,z,i] = 2*(y_train_encoded[:,z,i]-s_min[z,i])/srange[z,i]-1

    for i in range(12):
        y_valid_encoded[:,z,i] = 2*(y_valid_encoded[:,z,i]-s_min[z,i])/srange[z,i]-1
    
    for i in range(12):
        y_test_encoded[:,z,i] = 2*(y_test_encoded[:,z,i]-s_min[z,i])/srange[z,i]-1

In [22]:
# ----------------------------- numerically safe attention masks -----------------------------
def _to_additive_attn_mask(score: torch.Tensor, cap: float = 10.0):
    """
    Convert arbitrary 'score' to a numerically safe additive mask in [-cap, 0]:
      mask = score - max(score, dim=-1), so mask <= 0 everywhere,
      then clamp to [-cap, 0] to bound magnitude under AMP.
    score: [B, Lq, Lk] (float/half)
    returns: [B, Lq, Lk] (float32) in [-cap, 0]
    """
    s = score.detach().float()
    s = s - s.max(dim=-1, keepdim=True).values          # <= 0 per row
    s = s.clamp(-cap, 0.0)
    return s

def _safe_bias(mask: torch.Tensor, n_heads: int):
    """
    mask: [B, Lq, Lk] additive mask in [-cap, 0] (float32).
    Returns [B*nH, Lq, Lk] float32, repeated per head.
    """
    return mask.repeat_interleave(n_heads, dim=0)

# ----------------------------- transformer blocks -----------------------------
class EdgeAwareBlock(nn.Module):
    """Self-attention with additive geodesic/ohmic bias (safe for fp16)."""
    def __init__(self, d_model: int, n_heads: int = 8, dropout: float = 0.0, ff_mult: int = 4):
        super().__init__()
        self.n_heads = n_heads
        self.attn = nn.MultiheadAttention(d_model, n_heads, dropout=dropout, batch_first=True)
        self.ln1 = nn.LayerNorm(d_model)
        self.ln2 = nn.LayerNorm(d_model)
        self.ff  = nn.Sequential(nn.Linear(d_model, ff_mult * d_model), nn.GELU(),
                                 nn.Linear(ff_mult * d_model, d_model))
        self.drop = nn.Dropout(dropout)

    def forward(self, X: torch.Tensor, attn_bias: torch.Tensor):
        """
        X: [B,L,D]
        attn_bias: [B,L,L] float32, **must already be in [-cap, 0]** (use _to_additive_attn_mask upstream)
        """
        bias = _safe_bias(attn_bias, self.n_heads)       # [B*H,L,L] float32
        QKV = self.ln1(X)
        Y,_ = self.attn(QKV, QKV, QKV, attn_mask=bias)   # stable additive mask
        X = X + self.drop(Y)
        X = X + self.drop(self.ff(self.ln2(X)))
        return X

class EdgeAwareEncoder(nn.Module):
    def __init__(self, d_model: int, n_layers: int = 4, n_heads: int = 8, dropout: float = 0.0):
        super().__init__()
        self.layers = nn.ModuleList([EdgeAwareBlock(d_model, n_heads, dropout) for _ in range(n_layers)])
        self.out_ln = nn.LayerNorm(d_model)

    def forward(self, X: torch.Tensor, attn_mask: torch.Tensor):
        """
        X: [B,L,D]
        attn_mask: [B,L,L] float32, in [-cap, 0]
        """
        for layer in self.layers:
            X = layer(X, attn_mask)
        return self.out_ln(X)

class BipartiteCrossAttentionFusion(nn.Module):
    """Primary↔Secondary bidirectional cross-attention with coupling-aware mask."""
    def __init__(self, d_model: int, n_heads: int = 8, dropout: float = 0.0, k_top: int = 16):
        super().__init__()
        self.k_top = k_top
        self.h = n_heads
        self.attn_ps = nn.MultiheadAttention(d_model, n_heads, dropout=dropout, batch_first=True)  # P<-S
        self.attn_sp = nn.MultiheadAttention(d_model, n_heads, dropout=dropout, batch_first=True)  # S<-P
        self.ln_p1 = nn.LayerNorm(d_model); self.ln_s1 = nn.LayerNorm(d_model)
        self.ln_p2 = nn.LayerNorm(d_model); self.ln_s2 = nn.LayerNorm(d_model)
        self.ff_p  = nn.Sequential(nn.Linear(d_model, 4*d_model), nn.GELU(), nn.Linear(4*d_model, d_model))
        self.ff_s  = nn.Sequential(nn.Linear(d_model, 4*d_model), nn.GELU(), nn.Linear(4*d_model, d_model))
        self.drop  = nn.Dropout(dropout)

    def forward(self, Zp: torch.Tensor, Zs: torch.Tensor, feats_p7: torch.Tensor, feats_s7: torch.Tensor):
        # Build raw scores
        score_ps = build_intercoil_bias(feats_p7, feats_s7, k_top=self.k_top)  # [B,Lp,Ls] (fp32)
        score_sp = build_intercoil_bias(feats_s7, feats_p7, k_top=self.k_top)  # [B,Ls,Lp] (fp32)

        # Convert to **non-positive** additive masks (numerically stable under AMP)
        mask_ps = _to_additive_attn_mask(score_ps, cap=10.0)  # [-10, 0], [B,Lp,Ls]
        mask_sp = _to_additive_attn_mask(score_sp, cap=10.0)  # [-10, 0], [B,Ls,Lp]

        Mps = _safe_bias(mask_ps, self.h)   # [B*h, Lp, Ls]
        Msp = _safe_bias(mask_sp, self.h)   # [B*h, Ls, Lp]

        P, S = self.ln_p1(Zp), self.ln_s1(Zs)
        attn_ps,_ = self.attn_ps(P, S, S, attn_mask=Mps)  # P attends to S
        Zp = Zp + self.drop(attn_ps); Zp = Zp + self.drop(self.ff_p(self.ln_p2(Zp)))

        attn_sp,_ = self.attn_sp(S, P, P, attn_mask=Msp)  # S attends to P
        Zs = Zs + self.drop(attn_sp); Zs = Zs + self.drop(self.ff_s(self.ln_s2(Zs)))
        return Zp, Zs

# ----------------------------- main encoder -----------------------------
class SelfSupervisedGraphEncoder(nn.Module):
    """
    Produces a single sequence of tokens encoding: ports(primary/secondary), coupling latents,
    all segment tokens (P + S), and one global token. Role/coil/process embeddings disambiguate.
    """
    ROLE_SEGMENT, ROLE_PORT, ROLE_COUPLING, ROLE_GLOBAL = 0, 1, 2, 3
    COIL_NONE, COIL_PRIMARY, COIL_SECONDARY = 0, 1, 2

    def __init__(self,
                 d_model: int = 256,
                 num_ports_per_coil: int = 2,
                 n_coupling_tokens: int = 16,
                 n_fuse_heads: int = 8,
                 fuse_k_top: int = 16,
                 fuse_dropout: float = 0.0,
                 n_intra_layers: int = 4,
                 n_intra_heads: int = 8,
                 intra_dropout: float = 0.0,
                 post_concat_layers: int = 0,
                 post_concat_heads: int = 4,
                 post_concat_dropout: float = 0.0,
                 proc_dim: int = 4):
        super().__init__()
        self.d = d_model
        self.num_ports = num_ports_per_coil

        # We don't know enriched input dim until first call
        self.to_embed_p = nn.LazyLinear(d_model)
        self.to_embed_s = nn.LazyLinear(d_model)

        self.enc_p = EdgeAwareEncoder(d_model, n_layers=n_intra_layers, n_heads=n_intra_heads, dropout=intra_dropout)
        self.enc_s = EdgeAwareEncoder(d_model, n_layers=n_intra_layers, n_heads=n_intra_heads, dropout=intra_dropout)

        self.fusion = BipartiteCrossAttentionFusion(d_model, n_heads=n_fuse_heads,
                                                    dropout=fuse_dropout, k_top=fuse_k_top)

        self.port_pool_p = QueryPool(d_model, n_tokens=num_ports_per_coil, n_heads=4)
        self.port_pool_s = QueryPool(d_model, n_tokens=num_ports_per_coil, n_heads=4)
        self.couple_pool = CouplingBottleneck(d_model, n_tokens=n_coupling_tokens, n_heads=8)
        self.global_pool = QueryPool(d_model, n_tokens=1, n_heads=4)

        self.role_embed = nn.Embedding(4, d_model)
        self.coil_embed = nn.Embedding(3, d_model)
        self.proc_proj  = nn.Linear(proc_dim, d_model)

        self.post_layers = nn.ModuleList([
            nn.TransformerEncoderLayer(d_model=d_model, nhead=post_concat_heads,
                                       dim_feedforward=4*d_model, dropout=post_concat_dropout,
                                       batch_first=True, activation="gelu", norm_first=True)
            for _ in range(post_concat_layers)
        ])

    @staticmethod
    def _add_role_coil(X: torch.Tensor, role_id: int, coil_id: int,
                       role_embed: nn.Embedding, coil_embed: nn.Embedding, proc_tok: torch.Tensor = None):
        B, L, D = X.shape
        X = X + role_embed.weight[role_id].view(1, 1, D)
        X = X + coil_embed.weight[coil_id].view(1, 1, D)
        if proc_tok is not None:
            X = X + proc_tok.unsqueeze(1)
        return X

    @torch.no_grad()
    def _build_masks(self, B: int, sizes, device):
        Pp, Ps, C, Lp, Ls = sizes
        Ltot = Pp + Ps + C + Lp + Ls + 1
        masks = {
            "ports_mask":    torch.zeros(B, Ltot, dtype=torch.bool, device=device),
            "coupling_mask": torch.zeros(B, Ltot, dtype=torch.bool, device=device),
            "segP_mask":     torch.zeros(B, Ltot, dtype=torch.bool, device=device),
            "segS_mask":     torch.zeros(B, Ltot, dtype=torch.bool, device=device),
            "global_mask":   torch.zeros(B, Ltot, dtype=torch.bool, device=device),
        }
        slices = {}
        start = 0
        slices["ports_slice"]    = slice(start, start + Pp + Ps);  start += Pp + Ps
        masks["ports_mask"][:, slices["ports_slice"]] = True
        slices["coupling_slice"] = slice(start, start + C);        start += C
        masks["coupling_mask"][:, slices["coupling_slice"]] = True
        slices["segP_slice"]     = slice(start, start + Lp);       start += Lp
        masks["segP_mask"][:, slices["segP_slice"]] = True
        slices["segS_slice"]     = slice(start, start + Ls);       start += Ls
        masks["segS_mask"][:, slices["segS_slice"]] = True
        slices["global_slice"]   = slice(start, start + 1)
        masks["global_mask"][:, slices["global_slice"]] = True
        return masks, slices

    def forward(self, inp1_7: torch.Tensor, inp2_7: torch.Tensor, proc_vec: torch.Tensor = None):
        """
        inp1_7, inp2_7: [B, L, 7]   (primary, secondary)
        proc_vec: [B, 4] = [t_metal, h_sub, eps_r, rho_sheet] (optional; defaults provided)
        Returns: tokens [B, Ltot, D], masks dict, slices dict
        """
        B = inp1_7.size(0)
        dev = inp1_7.device
        if proc_vec is None:
            proc_vec = torch.tensor([[2e-6, 3e-6, 3.9, 0.05]], dtype=inp1_7.dtype, device=dev).repeat(B, 1)
        proc_tok = self.proc_proj(proc_vec)  # [B,D]

        # enrich & embed
        feats_p, _ = enrich_segment_features(inp1_7, proc=dict(
            t_metal=float(proc_vec[0,0].item()),
            h_sub=float(proc_vec[0,1].item()),
            eps_r=float(proc_vec[0,2].item()),
            rho_sheet=float(proc_vec[0,3].item())
        ))
        feats_s, _ = enrich_segment_features(inp2_7, proc=dict(
            t_metal=float(proc_vec[0,0].item()),
            h_sub=float(proc_vec[0,1].item()),
            eps_r=float(proc_vec[0,2].item()),
            rho_sheet=float(proc_vec[0,3].item())
        ))

        Zp = self.to_embed_p(feats_p)  # [B,Lp,D]
        Zs = self.to_embed_s(feats_s)  # [B,Ls,D]

        # -------- Intra-coil encoders with SAFE additive masks --------
        raw_p = build_ohmic_bias(inp1_7)                    # [B,Lp,Lp] (fp32, can be <= 0)
        raw_s = build_ohmic_bias(inp2_7)                    # [B,Ls,Ls]
        mask_p = _to_additive_attn_mask(raw_p, cap=10.0)    # [-10, 0]
        mask_s = _to_additive_attn_mask(raw_s, cap=10.0)    # [-10, 0]
        Zp = self.enc_p(Zp, mask_p)
        Zs = self.enc_s(Zs, mask_s)

        # -------- Cross-coil physics-aware fusion (SAFE masks inside) --------
        Zp, Zs = self.fusion(Zp, Zs, inp1_7, inp2_7)

        # pools
        ports_p  = self.port_pool_p(Zp)                  # [B,Pp,D]
        ports_s  = self.port_pool_s(Zs)                  # [B,Ps,D]
        coupling = self.couple_pool(Zp, Zs)              # [B,C,D]
        global_t = self.global_pool(torch.cat([Zp, Zs], dim=1))  # [B,1,D]

        # role/coil/process tagging
        Zp        = self._add_role_coil(Zp,       self.ROLE_SEGMENT,  self.COIL_PRIMARY,  self.role_embed, self.coil_embed, proc_tok)
        Zs        = self._add_role_coil(Zs,       self.ROLE_SEGMENT,  self.COIL_SECONDARY,self.role_embed, self.coil_embed, proc_tok)
        ports_p   = self._add_role_coil(ports_p,  self.ROLE_PORT,     self.COIL_PRIMARY,  self.role_embed, self.coil_embed, proc_tok)
        ports_s   = self._add_role_coil(ports_s,  self.ROLE_PORT,     self.COIL_SECONDARY,self.role_embed, self.coil_embed, proc_tok)
        coupling  = self._add_role_coil(coupling, self.ROLE_COUPLING, self.COIL_NONE,     self.role_embed, self.coil_embed, proc_tok)
        global_t  = self._add_role_coil(global_t, self.ROLE_GLOBAL,   self.COIL_NONE,     self.role_embed, self.coil_embed, proc_tok)

        # concatenate full sequence
        tokens = torch.cat([ports_p, ports_s, coupling, Zp, Zs, global_t], dim=1)  # [B,Ltot,D]
        for layer in self.post_layers:
            tokens = layer(tokens)

        sizes = (ports_p.size(1), ports_s.size(1), coupling.size(1), Zp.size(1), Zs.size(1))
        masks, slices = self._build_masks(B, sizes, device=tokens.device)
        return tokens, masks, slices


In [23]:
def custom_loss(y_true, y_pred):
    Lf = torch.sqrt(torch.mean((y_true - y_pred) ** 2, dim=1))  # Compute RMSE across feature axis
    return torch.log(torch.mean(Lf))  # Compute log of mean RMSE
    #return (torch.mean(Lf))

In [24]:
class GraphSSLDataset(Dataset):
    """
    Required:
      p_segs: [N, Lp, 7]
      s_segs: [N, Ls, 7]
    Optional (omit or pass None to skip related losses):
      proc:    [N,4]  -> [t_metal, h_sub, eps_r, rho_sheet]
      p_ports: [N,Pp,2]
      s_ports: [N,Ps,2]
      image:   [N,C,H,W]
      C_target:[N,2P,2P]
    """
    def __init__(self, p_segs, s_segs, proc=None, p_ports=None, s_ports=None, image=None, C_target=None):
        self.p_segs = p_segs
        self.s_segs = s_segs
        self.proc = proc
        self.p_ports = p_ports
        self.s_ports = s_ports
        self.image = image
        self.C_target = C_target
    def __len__(self): return self.p_segs.shape[0]
    def __getitem__(self, i):
        d = {"p_segs": self.p_segs[i], "s_segs": self.s_segs[i]}
        if self.proc is not None:     d["proc"] = self.proc[i]
        if self.p_ports is not None:  d["p_ports"] = self.p_ports[i]
        if self.s_ports is not None:  d["s_ports"] = self.s_ports[i]
        if self.image is not None:    d["image"] = self.image[i]
        if self.C_target is not None: d["C_target"] = self.C_target[i]
        return d

# If your data are NumPy arrays, convert them first:
# x_train_a: [N, Lp, 7], x_train_b: [N, Ls, 7]
p_segs = torch.from_numpy(x_train_a).float()
s_segs = torch.from_numpy(x_train_b).float()

# Optional extras (set to None if you don't have them yet)
proc     = None           # torch.tensor([...],[N,4]).float()
p_ports  = None           # torch.tensor([...],[N,Pp,2]).float()
s_ports  = None           # torch.tensor([...],[N,Ps,2]).float()
image    = None           # torch.tensor([...],[N,C,H,W]).float()
C_target = None           # torch.tensor([...],[N,2P,2P]).float()

dataset = GraphSSLDataset(p_segs, s_segs, proc, p_ports, s_ports, image, C_target)

In [25]:
# Repro/compute
SEED = 1337
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
AMP_ENABLED = (device.type == "cuda")

# Data loading
batch_size         = 32
val_frac           = 0.20
num_workers_train  = 4
num_workers_val    = 2
drop_last_train    = True
drop_last_val      = False

# Optimization & schedule
epochs        = 120
learning_rate = 1e-3
weight_decay  = 0.05
grad_clip     = 1.0
lr_decay_base   = 0.95
lr_decay_period = 20  # epochs-equivalent in the lambda
def lr_lambda(epoch): return lr_decay_base ** (epoch / lr_decay_period)

# Logging / checkpoints
PRINT_EVERY = 20
ckpt_path   = "best_ssl.pt"

# (Optional) SSL loss weights for SSLTrainer (if used elsewhere)
ssl_loss_weights = dict(reach=0.25, proxy=0.25, equiv=0.15, d4=0.15, C=0.10, clip=0.10)

In [26]:

# ========================== VALIDATION-ENABLED SSL TRAINING ==========================
PRINT_EVERY = 20  # print every 20 epochs

# --- split your dataset into train/val (80/20 by default) ---
val_frac = 0.2
N = len(dataset)
n_val = max(1, int(N * val_frac))
n_train = N - n_val
train_set, val_set = torch.utils.data.random_split(dataset, [n_train, n_val])

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True,
                                           drop_last=True, num_workers=4)
val_loader   = torch.utils.data.DataLoader(val_set,   batch_size=batch_size, shuffle=False,
                                           drop_last=False, num_workers=2)

# --- optimizer/scheduler as before ---
optimizer = optim.AdamW(ssl.parameters(), lr=learning_rate, weight_decay=weight_decay)
def lr_lambda(epoch): return 0.95 ** (epoch / 20)
scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)
scaler = torch.cuda.amp.GradScaler(enabled=device.type == "cuda")

def _agg_dict(keys):
    return {k: 0.0 for k in keys}

def _normalize_agg(d, denom):
    for k in d:
        d[k] /= max(1, denom)

def evaluate_ssl(model, loader):
    model.eval()
    keys = ["loss","reach","proxy","equiv","d4","C","clip"]
    agg = {k: 0.0 for k in keys}
    steps = 0
    skipped = 0
    with torch.no_grad():
        for batch in loader:
            for k, v in list(batch.items()):
                if torch.is_tensor(v):
                    batch[k] = v.to(device, non_blocking=True)
            loss, logs = model.forward_one(batch)
            if not torch.isfinite(loss):
                skipped += 1
                continue
            steps += 1
            for k in keys:
                val = logs.get(k, 0.0)
                val = float(val) if isinstance(val, (float, int)) else float(torch.nan_to_num(val, nan=0.0, posinf=0.0, neginf=0.0))
                agg[k] += val
    denom = max(1, steps)
    for k in agg:
        agg[k] /= denom
    # keep signature the same (dict), but include extra info as optional fields
    agg["steps"] = steps
    agg["skipped"] = skipped
    return agg

def train_ssl_model(model, optimizer, scheduler, num_epochs, train_loader, val_loader, ckpt_path="best_ssl.pt"):
    best_val = float("inf")
    keys = ["loss","reach","proxy","equiv","d4","C","clip"]

    for epoch in range(num_epochs):
        model.train()
        train_agg = {k: 0.0 for k in keys}
        steps = 0
        skipped_train = 0

        for batch in train_loader:
            for k, v in list(batch.items()):
                if torch.is_tensor(v):
                    batch[k] = v.to(device, non_blocking=True)

            with torch.cuda.amp.autocast(enabled=(device.type=="cuda")):
                loss, logs = model.forward_one(batch)

            if not torch.isfinite(loss):
                optimizer.zero_grad(set_to_none=True)
                skipped_train += 1
                continue

            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
            scaler.step(optimizer); scaler.update()
            optimizer.zero_grad(set_to_none=True)

            steps += 1
            for k in keys:
                val = logs.get(k, 0.0)
                val = float(val) if isinstance(val, (float, int)) else float(torch.nan_to_num(val, nan=0.0, posinf=0.0, neginf=0.0))
                train_agg[k] += val

        scheduler.step()
        denom = max(1, steps)
        for k in train_agg:
            train_agg[k] /= denom

        # ---- VALIDATION ----
        val_agg = evaluate_ssl(model, val_loader)
        val_total = float(val_agg.get("loss", float("inf")))
        val_steps = int(val_agg.get("steps", 0))
        val_skipped = int(val_agg.get("skipped", 0))

        # ---- CHECKPOINT ----
        if torch.isfinite(torch.tensor(val_total)) and (val_total < best_val):
            best_val = val_total
            torch.save({
                "model": model.state_dict(),
                "optimizer": optimizer.state_dict(),
                "scheduler": scheduler.state_dict(),
                "epoch": epoch+1,
                "best_val": best_val
            }, ckpt_path)

        # ---- PRINT: epoch 1, every PRINT_EVERY, and final epoch ----
        is_print_epoch = ((epoch + 1) % PRINT_EVERY == 0) or (epoch == 0) or ((epoch + 1) == num_epochs)
        if is_print_epoch:
            lr = optimizer.param_groups[0].get("lr", None)
            tr = " ".join([f"{k}:{train_agg[k]:.4f}" for k in keys])
            va = " ".join([f"{k}:{float(val_agg.get(k, 0.0)):.4f}" for k in keys])
            extras = []
            if lr is not None:
                extras.append(f"lr:{lr:.3e}")
            extras.append(f"steps:{steps} skipped:{skipped_train}")
            extras.append(f"val_steps:{val_steps} val_skipped:{val_skipped}")
            extra_str = "  [" + " | ".join(extras) + "]"
            print(f"[Epoch {epoch+1}/{num_epochs}]  TRAIN {tr}   |   VAL {va}   (best_val={best_val:.4f}){extra_str}", flush=True)

# --- kick off training with validation tracking ---
train_ssl_model(ssl, optimizer, scheduler, epochs, train_loader, val_loader, ckpt_path="best_ssl.pt")

/tmp/ipykernel_3789183/3731822080.py:20: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=device.type == "cuda")
/tmp/ipykernel_3789183/3731822080.py:64: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device.type=="cuda")):


[Epoch 1/120]  TRAIN loss:0.0000 reach:0.0000 proxy:0.0000 equiv:0.0000 d4:0.0000 C:0.0000 clip:0.0000   |   VAL loss:0.0000 reach:0.0000 proxy:0.0000 equiv:0.0000 d4:0.0000 C:0.0000 clip:0.0000   (best_val=0.0000)
[Epoch 20/120]  TRAIN loss:0.0000 reach:0.0000 proxy:0.0000 equiv:0.0000 d4:0.0000 C:0.0000 clip:0.0000   |   VAL loss:0.0000 reach:0.0000 proxy:0.0000 equiv:0.0000 d4:0.0000 C:0.0000 clip:0.0000   (best_val=0.0000)
[Epoch 40/120]  TRAIN loss:0.0000 reach:0.0000 proxy:0.0000 equiv:0.0000 d4:0.0000 C:0.0000 clip:0.0000   |   VAL loss:0.0000 reach:0.0000 proxy:0.0000 equiv:0.0000 d4:0.0000 C:0.0000 clip:0.0000   (best_val=0.0000)
[Epoch 60/120]  TRAIN loss:0.0000 reach:0.0000 proxy:0.0000 equiv:0.0000 d4:0.0000 C:0.0000 clip:0.0000   |   VAL loss:0.0000 reach:0.0000 proxy:0.0000 equiv:0.0000 d4:0.0000 C:0.0000 clip:0.0000   (best_val=0.0000)
[Epoch 80/120]  TRAIN loss:0.0000 reach:0.0000 proxy:0.0000 equiv:0.0000 d4:0.0000 C:0.0000 clip:0.0000   |   VAL loss:0.0000 reach:0.000

In [16]:
from torchinfo import summary
print(np.shape(x1))
summary(model, input_size=[(1, 51, 7),(1,51,7)])

NameError: name 'x1' is not defined